# PSI On SPU

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

PSI([Private Set Intersection](https://en.wikipedia.org/wiki/Private_set_intersection)) is a cryptographic technique that allows two parties holding sets to compare encrypted versions of these sets in order to compute the intersection. In this scenario, neither party reveals anything to the counterparty except for the elements in the intersection.

In SecretFlow, SPU device supports three PSI protocol:

- [ECDH](https://ieeexplore.ieee.org/document/6234849/)：semi-honest, based on public key encryption, suitable for small datasets.
- [KKRT](https://eprint.iacr.org/2016/799.pdf)：semi-host, based on cuckoo hashing and OT extension, suitable for large datasets.
- [BC22PCG](https://eprint.iacr.org/2022/334): semi-host, psi from pseudorandom correlation generators.

Before we start, we need to initialize the environment. The following three nodes `alice`, `bob`, and `carol` are created on a single machine to simulate multiple participants.

In [ ]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'carol'], address='local')

## Preparing dataset

First, we need a dataset for constructing vertical partitioned scenarios. For simplicity, we use [iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) dataset here. We add two columns to it for subsequent single-column and multi-column intersection demonstrations

- uid：Sample unique ID.
- month：Simulate a scenario where samples are generated monthly. The first 50% of the samples are generated in January, and the last 50% of the samples are generated in February.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris

data, target = load_iris(return_X_y=True, as_frame=True)
data['uid'] = np.arange(len(data)).astype('str')
data['month'] = ['Jan'] * 75 + ['Feb'] * 75

data

In the actual scenario, the sample data is provided by each participant, and the fields for intersection need to be agreed in advance:

- The intersection field can be single or multiple.
- The intersection field must be unique. If there is a duplicate, it needs to be deduplicated in advance.

For example, The following is the data provided by alice for PSI intersection, the intersection field is `uid` and `month`，we can see that [1, 'Jan'] is duplicated.
```
alice.csv
---------
uid   month   c0
1     Jan     5.8
2     Jan     5.4
1     Jan     5.8
1     Feb     7.4
```
The data after deduplication is
```
alice.csv
---------
uid   month   c0
1     Jan     5.8
2     Jan     5.4
1     Feb     7.4
```
We randomly sample the iris data three times to simulate the data provided by `alice`, `bob`, and `carol`, and the three data are in an unaligned state.


In [ ]:
import os

os.makedirs('.data', exist_ok=True)
da, db, dc = data.sample(frac=0.9), data.sample(frac=0.8), data.sample(frac=0.7)

da.to_csv('.data/alice.csv', index=False)
db.to_csv('.data/bob.csv', index=False)
dc.to_csv('.data/carol.csv', index=False)

## Two parties PSI

We virtualize three logical devices on the physical device:

- alice, bob: PYU device, responsible for the local plaintext computation of the participant.
- spu：SPUdevice, consists of alice and bob, responsible for the ciphertext calculation of the two parties.

In [ ]:
alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

### Single-column PSI

Next, we use `uid` to intersect the two data, SPU provide `psi` which take the csv file as input and generate the csv file after the intersection. The default protocol is KKRT.

In [ ]:
input_path = {alice.party: '.data/alice.csv', bob.party: '.data/bob.csv'}
output_path = {alice.party: '.data/alice_psi.csv', bob.party: '.data/bob_psi.csv'}
spu.psi({alice.party:["uid"], bob.party: ["uid"]}, input_path, output_path, 'alice', disable_alignment=True)

To check the correctness of the results, we use [pandas.DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) to inner join da and db. It can be seen that the two data have been aligned according to `uid` and sorted according to their lexicographical order.

In [ ]:
import pandas as pd

df:pd.DataFrame = da.join(db.set_index('uid'), on='uid', how='inner', rsuffix='_bob', sort=True)
expected = df[da.columns].astype({'uid': 'int64'}).sort_values(by='uid', ascending=True).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv').sort_values(by='uid', ascending=True).reset_index(drop=True)
db_psi = pd.read_csv('.data/bob_psi.csv').sort_values(by='uid', ascending=True).reset_index(drop=True)

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)


### Multi-columns PSI

We can also use multiple fields to intersect, the following demonstrates the use of `uid` and `month` to intersect two data. In terms of implementation, multiple fields are concatenated into a string, so please ensure that there is no duplication of the multi-column composite primary key.

In [ ]:
spu.psi({alice.party:['uid', 'month'], bob.party:['uid', 'month']}, input_path, output_path, 'alice')

Similarly, we use pandas.DataFrame.join to verify the correctness of the result, we can see that the two data have been aligned according to `uid` and `month`, and sorted according to their lexicographical order.

In [ ]:
df:pd.DataFrame = da.join(
    db.set_index(['uid', 'month']),
    on=['uid', 'month'],
    how='inner',
    rsuffix='_bob',
    sort=True,
)
expected = df[da.columns].astype({'uid': 'int64'}).sort_values(by='uid', ascending=True).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv').sort_values(by='uid', ascending=True).reset_index(drop=True)
db_psi = pd.read_csv('.data/bob_psi.csv').sort_values(by='uid', ascending=True).reset_index(drop=True)

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)

## Three parties PSI

Next, we add a third-party `carol`, and create a PYU device for it, as well as an SPU device built by the third party.

In [ ]:
carol = sf.PYU('carol')
spu_3pc = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob', 'carol']))

Then, use `uid` and `month` as the composite primary key to perform a three-way negotiation. It should be noted that the three-way negotiation only supports the ECDH protocol for the time being.

In [ ]:
input_path = {alice.party: '.data/alice.csv', bob.party: '.data/bob.csv', carol.party: '.data/carol.csv'}
output_path = {
    alice.party: '.data/alice_psi.csv',
    bob.party: '.data/bob_psi.csv',
    carol.party: '.data/carol_psi.csv',
}
keys = ['uid', 'month']
spu_3pc.psi(
    {alice.party:keys, bob.party:keys, carol.party:keys}, input_path, output_path, 'alice', protocol='PROTOCOL_ECDH_3PC'
)

Similarly, we use pandas.DataFrame.join to verify the correctness of the result.

In [ ]:
keys = ['uid', 'month']
df = da.join(db.set_index(keys), on=keys, how='inner', rsuffix='_bob', sort=True).join(
    dc.set_index(keys), on=keys, how='inner', rsuffix='_carol', sort=True
)
expected = df[da.columns].astype({'uid': 'int64'}).reset_index(drop=True)

da_psi = pd.read_csv('.data/alice_psi.csv')
db_psi = pd.read_csv('.data/bob_psi.csv')
dc_psi = pd.read_csv('.data/carol_psi.csv')

pd.testing.assert_frame_equal(da_psi, expected)
pd.testing.assert_frame_equal(db_psi, expected)
pd.testing.assert_frame_equal(dc_psi, expected)

## What's Next

OK! Through this tutorial, we have seen how to do two-party and three-party data intersections via SPU. After completing the data intersection, we can perform machine learning modeling on the aligned dataset.

- [Logistic Regression On SPU](./lr_with_spu.ipynb): Logistic regression modeling on SPU using JAX.
- [Neural Network on SPU](./nn_with_spu.ipynb): Neural Network Modeling on SPU with JAX.
- [Basic Split Learning](./split_learning_gnn.ipynb): Neural Network Modeling with TensorFlow and Split Learning.